In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

In [2]:
%matplotlib

Using matplotlib backend: Qt5Agg


# Advección

$$
\frac{\partial \psi}{\partial t} + \nabla\cdot\left(\psi\vec{u}\right) = 0
$$

# Lax Wendroff
$$
\frac{\partial \psi}{\partial t} + \frac{\partial f(\psi)}{\partial x} = 0
$$

$$
\psi_i^{n+1} = \psi_i^{n} - \frac{\Delta t}{2\Delta x}A[\psi_{i+1}^{n}-\psi_{i-1}^{n}] + \frac{\Delta t^2}{2\Delta x^2}A^2[\psi_{i+1}^{n}-2\psi_i^n+\psi_{i-1}^{n}]
$$

# Upwind
$$
\psi_i^{n+1} = \psi_i^{n} - \frac{\Delta t}{\Delta x}\left[a^+\left(\psi_i^n - \psi_{i - 1}^n\right) + a^-\left(\psi_{i + 1}^n - \psi_{i}^n\right)\right]
$$

## Convention
$$
\begin{matrix}
f(\psi) & = & u_x\psi \\
A & = & u_x \\
a^+ & = & \text{max}(a, 0) \\
a^- & = & \text{min}(a, 0)
\end{matrix}
$$

In [3]:
def lax_wendroff(psi: np.array, dt: float, dx: float, speed: float, stop_time: float, store_every: int = 1):
    def step(actual: np.array, alpha: float, beta: float) -> np.array:
        actual[1:-1] = actual[1:-1] - alpha * (actual[2:] - actual[:-2]) + beta * (actual[2:] - 2 * actual[1:-1] + actual[:-2])

        return actual
    
    if 0.5 * dx / dt < abs(speed):
        raise(ValueError(f"Speed {abs(speed)} is greater than information propagation speed {0.5 * dx / dt}"))
        
    alpha = dt / dx * 0.5 * speed
    beta = 0.5 * (dt * speed / dx) ** 2
    
    output = []
    extended = np.zeros(psi.shape[0] + 2)
    extended[1:-1] = psi
        
    for i in range(int(stop_time // dt)):
        if i % store_every == 0:
            output.append(
                {
                    'time': i * dt,
                    'data': extended[1:-1].copy()
                }
            )
            
        extended = step(extended, alpha, beta)
        
    return output

def upwind(psi: np.array, dt: float, dx: float, speed: float, stop_time: float, store_every: int = 1):
    def step(actual: np.array, advance_rate: float, a_p: float, a_m: float) -> np.array:
        actual[1:-1] -= (
            a_p * (actual[1:-1] - actual[:-2]) + 
            a_m * (actual[2:] - actual[1:-1])
        ) / advance_rate

        return actual
    
    advance_rate = dx / dt
    
    if advance_rate < abs(speed):
        raise(ValueError(f"Speed {abs(speed)} is greater than information propagation speed {advance_rate}"))
    
    output = []
    extended = np.zeros(psi.shape[0] + 2)
    extended[1:-1] = psi
    
    a_p = max(speed, 0)
    a_m = min(speed, 0)
    
    for i in range(int(stop_time // dt)):
        if i % store_every == 0:
            output.append(
                {
                    'time': i * dt,
                    'data': extended[1:-1].copy()
                }
            )
            
        extended = step(
            extended,
            advance_rate=advance_rate,
            a_p=a_p,
            a_m=a_m
        )
        
    return output

def animate(x: np.array, data: dict):
    def anim(i):
        for key in data:
            lines[key].set_data(x, data[key][i]['data'])

        time = data[ref_key][i]['time']
        time_text.set_text(f"$t = {time: .2e}$")

        return (
            time_text,
            *lines.values()
        )
    
    fig, ax = plt.subplots()

    lines = {
        key: ax.plot([], [], label=key)[0]
        for key in data
    }
    
    ref_key = next(iter(lines.keys()))
    
    time_text = ax.text(
        0.5,
        0.85,
        "",
        transform=ax.transAxes,
        ha="center"
    )

    ax.legend()
    ax.set_xlim(x.min(), x.max())
    
    y_min = data[ref_key][0]['data'].min()
    y_max = data[ref_key][0]['data'].max()
    
    y_range = 0.05 * (y_max - y_min)
    
    ax.set_ylim(
        y_min - y_range,
        y_max + y_range
    )
    ax.grid()
    
    ax.set_ylabel("$\Psi(x, t)$")
    ax.set_xlabel("$x$")

    return FuncAnimation(
        fig,
        anim,
        frames=len(data[ref_key]),
        interval=len(data[ref_key]) / 24,
        blit=True
    )

In [4]:
n = 1000
x = np.linspace(0, 1, n)


cube_psi = np.zeros(n)
cube_psi[: int(n * (.05))] = 1

sin = np.sin(2 * np.pi * x / (x.max() - x.min()) * 2)

kwargs = dict(
    dx=x[1]-x[0],
    dt=1e-2,
    speed=0.05,
    stop_time=(x.max() - x.min()) / 0.05,
    store_every=5
)

In [5]:
cube = animate(
    x,
    {
        'Lax-Wendroff': lax_wendroff(psi=cube_psi, **kwargs),
        'Upwind': upwind(psi=cube_psi, **kwargs),
    }
)

cube.save("cube.mp4", writer='ffmpeg')

In [6]:
soft = animate(
    x,
    {
        'Lax-Wendroff': lax_wendroff(psi=sin, **kwargs),
        'Upwind': upwind(psi=sin, **kwargs),
    }
)

soft.save("soft.mp4", writer='ffmpeg')